In [1]:
import itertools
import pyhornedowl

## Open an ontology





In [2]:
%%time

onto = pyhornedowl.open_ontology("chebi_core.owl")

print (f"Loaded ontology has {len(onto.get_classes())} classes.")
print (f"Loaded ontology has {len(onto.get_axioms())} axioms.")


Loaded ontology has 185295 classes.
Loaded ontology has 2369426 axioms.
CPU times: user 26.8 s, sys: 1.26 s, total: 28.1 s
Wall time: 28.2 s


## Explore content

### Annotations and IDs

In [7]:
%%time

RDFSLABEL = "http://www.w3.org/2000/01/rdf-schema#label"
DEFN = "http://purl.obolibrary.org/obo/IAO_0000115"

# add a prefix mapping
onto.add_prefix_mapping("CHEBI","http://purl.obolibrary.org/obo/CHEBI_")

for label in ["caffeine","isocitric acid","is enantiomer of"]:
    iri = onto.get_iri_for_label(label)
    print(f"Got IRI: {iri} for label '{label}'.")
    print(f"Got label: '{onto.get_annotation(iri,RDFSLABEL)}' for IRI {iri}.")
    print(f"Got definition '{onto.get_annotation(iri,DEFN)}' for IRI {iri}")
    id = onto.get_id_for_iri(iri)
    print(f"Got ID {id} for IRI {iri}")
    

Got IRI: http://purl.obolibrary.org/obo/CHEBI_27732 for label 'caffeine'.
Got label: 'caffeine' for IRI http://purl.obolibrary.org/obo/CHEBI_27732.
Got definition 'A trimethylxanthine in which the three methyl groups are located at positions 1, 3, and 7. A purine alkaloid that occurs naturally in tea and coffee.' for IRI http://purl.obolibrary.org/obo/CHEBI_27732
Got ID CHEBI:27732 for IRI http://purl.obolibrary.org/obo/CHEBI_27732
Got IRI: http://purl.obolibrary.org/obo/CHEBI_30887 for label 'isocitric acid'.
Got label: 'isocitric acid' for IRI http://purl.obolibrary.org/obo/CHEBI_30887.
Got definition 'A tricarboxylic acid that is propan-1-ol with a hydrogen at each of the 3 carbon positions replaced by a carboxy group.' for IRI http://purl.obolibrary.org/obo/CHEBI_30887
Got ID CHEBI:30887 for IRI http://purl.obolibrary.org/obo/CHEBI_30887
Got IRI: http://purl.obolibrary.org/obo/chebi#is_enantiomer_of for label 'is enantiomer of'.
Got label: 'is enantiomer of' for IRI http://purl.obo

### Subclasses and descendents

In [8]:
%%time

parent_label = "diterpenoid"
parent_iri = onto.get_iri_for_label(parent_label)

descs = pyhornedowl.get_descendants(onto,parent_iri)

print(f"Got {len(descs)} descendents for class {parent_iri}, which has {len(onto.get_subclasses(parent_iri))} direct subclasses. Showing first 5: ")

for desc in itertools.islice(descs, 5): 
    print(f"Descendent: '{onto.get_annotation(desc,RDFSLABEL)}', IRI: {desc}")

Got 1589 descendents for class http://purl.obolibrary.org/obo/CHEBI_23849, which has 628 direct subclasses. Showing first 5: 
Descendent: '4-Keto-4'-hydroxyalloxanthin', IRI: http://purl.obolibrary.org/obo/CHEBI_185779
Descendent: 'gibberellin A3', IRI: http://purl.obolibrary.org/obo/CHEBI_28833
Descendent: 'Rabdoternin E', IRI: http://purl.obolibrary.org/obo/CHEBI_67680
Descendent: '19-(3-methyl-butanoyloxy)-villanovane-13alpha,17-diol', IRI: http://purl.obolibrary.org/obo/CHEBI_194566
Descendent: 'dehydroabietic acid', IRI: http://purl.obolibrary.org/obo/CHEBI_29571
CPU times: user 1.2 ms, sys: 389 µs, total: 1.59 ms
Wall time: 1.59 ms


### Superclasses and ancestors


In [9]:
%%time

child_label = "caffeine"
child_iri = onto.get_iri_for_label(child_label)

ancs = pyhornedowl.get_ancestors(onto,child_iri)

print(f"Got {len(ancs)} ancestors for class {child_iri} \
      which has {len(onto.get_superclasses(child_iri))} direct superclasses. Showing first 5:")

for anc in itertools.islice(ancs, 5): 
    print(f"Ancestor: '{onto.get_annotation(anc,RDFSLABEL)}', IRI: {anc}")




Got 34 ancestors for class http://purl.obolibrary.org/obo/CHEBI_27732       which has 2 direct superclasses. Showing first 5:
Ancestor: 'chemical entity', IRI: http://purl.obolibrary.org/obo/CHEBI_24431
Ancestor: 'heterocyclic compound', IRI: http://purl.obolibrary.org/obo/CHEBI_5686
Ancestor: 'p-block molecular entity', IRI: http://purl.obolibrary.org/obo/CHEBI_33675
Ancestor: 'heteropolycyclic compound', IRI: http://purl.obolibrary.org/obo/CHEBI_33671
Ancestor: 'organic heterobicyclic compound', IRI: http://purl.obolibrary.org/obo/CHEBI_27171
CPU times: user 243 µs, sys: 141 µs, total: 384 µs
Wall time: 426 µs


## Working with axioms

### Annotations


In [4]:
%%time

SMILES = "http://purl.obolibrary.org/obo/chebi/smiles"
CHARGE = "http://purl.obolibrary.org/obo/chebi/charge"

print (f"Getting all classes with charge and no smiles")
# Get all classes with asserted charge but no smiles
classes_with_charge = set()
classes_with_smiles = set()
i = 0
j = 0
for a in onto.get_axioms():
    #Example: ['AnnotationAssertion', 'http://purl.obolibrary.org/obo/CHEBI_27732', 'http://www.geneontology.org/formats/oboInOwl#hasAlternativeId', 'CHEBI:41472']
    if isinstance(a.axiom, pyhornedowl.model.AnnotationAssertion):

        if str(a.axiom.ann.ap.inner) == CHARGE:
            chg = str(a.axiom.ann.av)
            classes_with_charge.add(str(a.axiom.subject))
            i += 1
        if str(a.axiom.ann.ap.inner)==SMILES:
            smls = str(a.axiom.ann.av)
            classes_with_smiles.add(str(a.axiom.subject))
            j += 1
classes_with_charge_no_smiles = [c for c in classes_with_charge if c not in classes_with_smiles]
print(f"The ontology has {len(classes_with_charge_no_smiles)} classes with charge and no smiles.")

(i,j)

Getting all classes with charge and no smiles
The ontology has 2129 classes with charge and no smiles.
CPU times: user 5.83 s, sys: 5.5 s, total: 11.3 s
Wall time: 11.3 s


(154966, 152853)

### Object properties

In [6]:
%%time

HAS_PART = "http://purl.obolibrary.org/obo/BFO_0000051"

# Get all the asserted parts:
print("Getting all asserted part relations")
asserted_parts = set() # Will hold tuples
for a in onto.get_axioms():
    # Example: ['SubClassOf', 'http://purl.obolibrary.org/obo/CHEBI_27732', ['ObjectSomeValuesFrom', 'http://purl.obolibrary.org/obo/RO_0000087', 'http://purl.obolibrary.org/obo/CHEBI_85234']]
    if isinstance(a.axiom, pyhornedowl.model.SubClassOf) and \
      isinstance(a.axiom.sup, pyhornedowl.model.ClassExpression_ObjectSomeValuesFrom) and \
      str(a.axiom.sup.ope) ==HAS_PART:
        asserted_parts.add((a.axiom.sub,a.axiom.sup.bce))

print(f"There are {len(asserted_parts)} parthood relations asserted in ChEBI.")



Getting all asserted part relations


AttributeError: module 'model' has no attribute 'ClassExpression_ObjectSomeValuesFrom'

## Modifying the ontology

In [9]:
%%time

clssname = "caffeine"
clssid = onto.get_iri_for_label(clssname)

onto.add_axiom(['SubClassOf',clssid,'http://testing.testing'])
onto.add_axiom(['AnnotationAssertion', 'http://purl.obolibrary.org/obo/CHEBI_27732', 'http://test_annotation', 'Testing Added Annotation Value'])
print (f"Ontology now has {len(onto.get_axioms())} axioms.")

print ("Removing one axiom")
onto.remove_axiom(['SubClassOf', 'http://purl.obolibrary.org/obo/CHEBI_27732', 'http://purl.obolibrary.org/obo/CHEBI_27134'])
print (f"Ontology now has {len(onto.get_axioms())} axioms.")

print ("Adding a complex axiom")

onto.add_axiom(['SubClassOf', 'http://purl.obolibrary.org/obo/CHEBI_27732', ['ObjectSomeValuesFrom', 'http://test-op', 'http://test-ob-target']])

onto.add_axiom(['SubClassOf', 'http://purl.obolibrary.org/obo/CHEBI_27732', ['ObjectIntersectionOf', 'http://purl.obolibrary.org/obo/CHEBI_27134', 'http://test-ob-target']])


print (f"Ontology now has {len(onto.get_axioms())} axioms.")

print(f"Class {clssid} ({clssname}) has axioms: {onto.get_axioms_for_iri(clssid)}")


Ontology now has 1965686 axioms.
Removing one axiom
Ontology now has 1965685 axioms.
Adding a complex axiom
Ontology now has 1965687 axioms.
Class http://purl.obolibrary.org/obo/CHEBI_27732 (caffeine) has axioms: [['DeclareClass', 'http://purl.obolibrary.org/obo/CHEBI_27732'], ['SubClassOf', 'http://purl.obolibrary.org/obo/CHEBI_27732', 'http://purl.obolibrary.org/obo/CHEBI_26385'], ['SubClassOf', 'http://purl.obolibrary.org/obo/CHEBI_27732', 'http://testing.testing'], ['SubClassOf', 'http://purl.obolibrary.org/obo/CHEBI_27732', ['ObjectIntersectionOf', 'http://purl.obolibrary.org/obo/CHEBI_27134', 'http://test-ob-target']], ['SubClassOf', 'http://purl.obolibrary.org/obo/CHEBI_27732', ['ObjectSomeValuesFrom', 'http://purl.obolibrary.org/obo/RO_0000087', 'http://purl.obolibrary.org/obo/CHEBI_25435']], ['SubClassOf', 'http://purl.obolibrary.org/obo/CHEBI_27732', ['ObjectSomeValuesFrom', 'http://purl.obolibrary.org/obo/RO_0000087', 'http://purl.obolibrary.org/obo/CHEBI_35337']], ['SubClas

## Saving an ontology

In [28]:
%%time

onto.save_to_file("chebi-updated.owx")

CPU times: user 30.9 s, sys: 4min 17s, total: 4min 48s
Wall time: 5min 15s


## Opening an ontology from a string downloaded from a URL


In [10]:
%%time

from urllib.request import urlopen

ontourl = "https://raw.githubusercontent.com/addicto-org/addiction-ontology/master/addicto-merged.owx"
data = urlopen(ontourl).read() #bytes
ontostring = data.decode('utf-8')

onto2 = pyhornedowl.open_ontology(ontostring)

print (f"Loaded ontology has {len(onto2.get_classes())} classes.")
print (f"Loaded ontology has {len(onto2.get_axioms())} axioms.")

Loaded ontology has 772 classes.
Loaded ontology has 7251 axioms.
CPU times: user 151 ms, sys: 38.2 ms, total: 189 ms
Wall time: 1.18 s
